## Sim1 experiments

Take a very crude heating equation and create a dataset

Let $a(t)$ be the inside temperature at time $t$, $e(t)$ outside temperature, and $c(t) \in \{0, 1\}$ heating commands. A very crude heating equation is:
A very crude heating equation is:
$$
\begin{align*}
a(t)=&a(t-1)+\\
&\alpha_e \left( (k_e \circ e) [t-T_e] - a(t-1) \right) +\\
&\alpha_c (H_c - a(t-1))(k_c \circ c)[t-T_c]  
\end{align*}
$$
The $\circ$ is the convolution operator, $[]$ indexes into the array, $k_e$ and $k_c$ are two Gaussian kernels, $\alpha_e$ and $\alpha_c$ are scaling values and $T_e$, $T_c$ are some delays. Note that the heating agent is always at the same temperature $H_c$ and gradually heats up the air, with a delay $T_c$.

That is, the current temperature is the previous temperature plus the contributions from outside and from heating. The heat goes from the hot to cold areas, faster if the differences are higher. For the heating element, the source temperature is fixed and when the heater is off ($c(t) == 0$ it does not represent a source of 0 degree temperature, but its contributions to the system are zero.

In [ ]:
# Uncomment next line to get interactive graphics
%matplotlib widget   
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from fastai2.vision.all import *
import torch
import network_definitions
import seaborn as sns
import sim1_datagen
import dataprocessing

## Let's generate some data and save it

In [ ]:
Path("data").mkdir(exist_ok=True)
simulated_data_file = Path("data/sim1_data.csv")
if not simulated_data_file.exists():
    np.random.seed(42)
    N = 360 * sim1_datagen.ONE_DAY_MINUTES 
    F = 14
    outside, commands, result = sim1_datagen.generate_heating_data(N)
    features = sim1_datagen.generate_feature_block(outside, commands, result, F)
    date_index = pd.date_range(start='2018-01-01 00:00', periods=N, freq='min')
    sim1_df = pd.DataFrame(data=features, index=date_index, columns=[f"C{k}" for k in range(F)])
    sim1_df.to_csv(simulated_data_file, index=True, index_label='index')
else:
    sim1_df = pd.read_csv(simulated_data_file, index_col="index", parse_dates=True)
OUT_ID = 8
CMDS = 12
INTERIOR = 13
print(f"The shape of feature set: {sim1_df.shape}")
sim1_df.head()

In [ ]:
fig, ax = plt.subplots(3,1,figsize=(8, 4))
some_index = np.arange(0, sim1_df.shape[0])
ax[0].plot(some_index, sim1_df.iloc[:, OUT_ID])
ax[0].set_title("Outside temperature")
ax[1].plot(some_index, sim1_df.iloc[:, CMDS])
ax[1].set_title("Commands")
ax[2].plot(some_index, sim1_df.iloc[:, INTERIOR])
ax[2].set_title("Interior temperature")
fig.tight_layout()
fig.savefig("images/sim1_all_samples.png")

In [ ]:
past_length = 100
future_length = 60
batch_size = 8
sample_size = past_length +future_length
no_features = sim1_df.shape[1]

samples = sim1_datagen.generate_samples(sim1_df, [(sim1_df.index[0], sim1_df.index[-1])], aggregate_interval_mins=3,
                                        stride_step=0.1, sample_size=sample_size)
sample_count = len(samples)
ts = int(sample_count*0.9) # train/valid split
train_set = samples[0:ts]
valid_set = samples[ts:]
print(f"Train samples count: {len(train_set)}. Shape of a sample: {train_set[0].shape}, valid samples count: {len(valid_set)}")

itemizer = dataprocessing.HeatingItemiser(future_length)
itemizerT = dataprocessing.HeatingItemiser(future_length, transformer=True)
tls_train = TfmdLists(train_set, itemizer)
tls_valid = TfmdLists(valid_set, itemizer)
dloader = DataLoaders.from_dsets(tls_train, tls_valid, bs=batch_size).cuda()
tls_trainT = TfmdLists(train_set, itemizerT)
tls_validT = TfmdLists(valid_set, itemizerT)
dloaderT = DataLoaders.from_dsets(tls_trainT, tls_validT, bs=batch_size).cuda()

callbacks = [ReduceLROnPlateau(patience=5, factor=2),
             EarlyStoppingCallback(patience=15),
             SaveModelCallback(fname="best_model")]

In [ ]:
model_Linear= network_definitions.LinearModel(in_features=no_features, past_steps=past_length, future_steps=future_length).cuda()
learner_Linear = Learner(dloader, model_Linear, loss_func=MSELossFlat())
# print(learner_Linear.summary())
# learner_Linear.lr_find()
learner_Linear.fit_one_cycle(10, 5e-3, cbs=callbacks)
fig = plt.figure(figsize=(4, 2))  # because jupyter widgets always changing semantics.
learner_Linear.recorder.plot_loss()

In [ ]:
model_Cnn= network_definitions.CnnFcn(in_features=no_features, past_steps=past_length, future_steps=future_length, cnn_stack_len=1,
                                fcn_stack_len=1, cnn_filters=64, avg_pool_len=10, fcn_ratio=0.8).cuda()
learner_Cnn = Learner(dloader, model_Cnn, loss_func=MSELossFlat())
# print(learner_Cnn.summary())
# learner_Cnn.lr_find()
learner_Cnn.fit_one_cycle(10, 5e-4, cbs=callbacks)
fig = plt.figure(figsize=(4, 2))
learner_Cnn.recorder.plot_loss()

In [ ]:
model_EncDec= network_definitions.EncoderDecoder(in_features=no_features, past_steps=past_length, future_steps=future_length, 
                                                hidden_size=128, num_layers=2).cuda()
learner_EncDec = Learner(dloader, model_EncDec, loss_func=MSELossFlat())
# print(learner_EncDec.summary())
# learner_EncDec.lr_find()
learner_EncDec.fit_one_cycle(10, 5e-3, cbs=callbacks)
fig = plt.figure(figsize=(4, 2))
learner_EncDec.recorder.plot_loss()

In [ ]:
model_Att= network_definitions.AttentionModel(in_features=no_features, past_steps=past_length, future_steps=future_length, 
                                                hidden_size=128, num_layers=2).cuda()
learner_Att = Learner(dloader, model_Att, loss_func=MSELossFlat())
# print(learner_Att.summary())
# learner_Att.lr_find()
learner_Att.fit_one_cycle(5, 5e-4, cbs=callbacks)
fig = plt.figure(figsize=(4, 2))
learner_Att.recorder.plot_loss()

In [ ]:
model_TransS= network_definitions.SimpleTransformer(in_features=no_features, past_steps=past_length, future_steps=future_length, 
                                                d_model=128, dim_feedforward=512).cuda()
learner_TransS = Learner(dloader, model_TransS, loss_func=MSELossFlat())
# print(learner_TransS.summary())
# learner_TransS.lr_find()
learner_TransS.fit_one_cycle(5, 1e-4, cbs=callbacks)
fig = plt.figure(figsize=(4, 2))
learner_TransS.recorder.plot_loss()

In [ ]:
model_TransF= network_definitions.TransformerFull(in_features=no_features, past_steps=past_length, future_steps=future_length, 
                                                  target_features=2, d_model=128, dim_feedforward=512).cuda()
learner_TransF = Learner(dloaderT, model_TransF, loss_func=MSELossFlat())
# print(learner_TransF.summary())
# learner_TransF.lr_find()
learner_TransF.fit_one_cycle(5, 1e-4, cbs=callbacks)
fig = plt.figure(figsize=(4, 2))
learner_TransF.recorder.plot_loss()

In [ ]:
## Let's view some results!

In [ ]:
def get_lerner_prediction(learner, sample, label):
    decoded_output, _, _ = learner.predict(sample, with_input=False)
    return (decoded_output, label)

item = valid_set[167]

predictions =[
    get_lerner_prediction(learner_Linear, item, "Linear"),
    get_lerner_prediction(learner_Cnn, item, "CnnFcn"),
    get_lerner_prediction(learner_EncDec, item, "Encoder-Decoder"),
    get_lerner_prediction(learner_Att, item, "Attention"),
    get_lerner_prediction(learner_TransS, item, "TransS"),
    get_lerner_prediction(learner_TransF, item, "TransF"),
]
gt = item[-future_length:, INTERIOR]
cmds = item[-future_length:, CMDS]
# fig, ax = plt.subplots(2, 1, figsize=(5, 3))
# ax[0].plot(item[0].T)
# ax[0].set_title("x1 features")
# ax[1].plot(item[1])
# ax[1].set_title("x2, knwon future")
# fig.tight_layout()
fig, ax = plt.subplots(1, 1, figsize=(8, 4));ax=[ax]
ax[0].plot(gt, 'r-.', label="GT",)
for preds in predictions:
    ax[0].plot(preds[0], label=preds[1])
ax[0].legend(loc='upper left')
ax[0].set_title("Predictions for each model")
ax2 =  ax[0].twinx()
ax2.plot(cmds, 'k')
ax2.set_ylim([0, 4])
fig.tight_layout()
# fig.savefig("images/sim1_sample_training.png")

## Details on sim1 data generator

We will look at a small sample to see how the temperatures affect the end result.

In [ ]:
N = 10 * sim1_datagen.ONE_DAY_MINUTES
F = 14
outside, commands, result = sim1_datagen.generate_heating_data(N, command_seed=323)
# corectat cand heating e 0 sau 1 !!!!

n = 3200
l = 5000
fig, ax = plt.subplots(3,1,figsize=(8, 4))
ax[0].plot(outside[n:n+l])
ax[0].set_title("Temperature outside")
ax[1].plot(commands[n:n+l])
ax[1].set_title("Heating commands")
ax[2].plot(result[n:n+l])
ax[2].set_title("Inside temperature")
fig.tight_layout()
# fig.savefig("images/sim1_small_sample.png")
